<a href="https://colab.research.google.com/github/romybeaute/ModNeuro-MVA/blob/main/oddball_experiment_mental_action.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Code for experimental task : oddball condition

@author : Romy BEAUTÉ
romybeaute.univ@gmail.com


----------

This code is a support for the project in "Modeling in Neurosciences & elsewhere", in order to compare the results from the simulated agent (generative model) with pilot results in experimental context.
This script 1) generate oddball sounds and 2) launch the experimental task. 
Sounds of the Local Global experiments (Chennu, S. et al., 2013 & Bekinschtein, TA et al., 2009) are also available, in order to reproduce the task with different types of sounds (control). 

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

!pip install pygame psychopy


#Import libraries:
import pygame
from pygame.locals import *
import numpy as np
import pandas as pd
import random
import time
#from MCP_communication import *


import numpy as np
import pandas as pd
import random
import time
import os

     |████████████████████████████████| 21.8 MB 1.8 MB/s 
     |████████████████████████████████| 25.9 MB 3.0 MB/s 
     |████████████████████████████████| 4.0 MB 38.6 MB/s 
     |████████████████████████████████| 206 kB 59.0 MB/s 
     |████████████████████████████████| 11.0 MB 34.2 MB/s 
     |████████████████████████████████| 90 kB 9.6 MB/s 
     |████████████████████████████████| 5.8 MB 46.8 MB/s 
     |████████████████████████████████| 45 kB 2.7 MB/s 
     |████████████████████████████████| 122 kB 48.3 MB/s 
     |████████████████████████████████| 181 kB 45.5 MB/s 
     |████████████████████████████████| 47 kB 4.2 MB/s 
     |████████████████████████████████| 978 kB 42.3 MB/s 
     |████████████████████████████████| 2.1 MB 31.2 MB/s 
     |████████████████████████████████| 29.1 MB 1.1 MB/s 
     |████████████████████████████████| 32.8 MB 57 kB/s 
     |████████████████████████████████| 131 kB 64.0 MB/s 
     |████████████████████████████████| 54 kB 2.5 MB/s 
     |████████████████

pygame 2.1.2 (SDL 2.0.16, Python 3.7.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


### HELPER CONDITIONS 1

In [2]:
# set PATH=/home/romy.beaute/anaconda3/envs/mne
# conda activate psychopy
# conda install numpy scipy matplotlib pandas pyopengl pillow lxml openpyxl xlrd configobj pyyaml gevent greenlet msgpack-python psutil pytables requests[security] cffi seaborn wxpython cython future pyzmq pyserial
# conda install -c conda-forge pyglet pysoundfile python-bidi moviepy pyosf
# pip install zmq json-tricks pyparallel sounddevice pygame pysoundcard psychopy_ext psychopy

"""
Created on Feb 20/2022

@author: Romy Beauté
romybeaute.univ@gmail.com

LG and oddball sounds for comparing human and simulated agent responses task - PILOT


"""

from __future__ import division

import contextlib

import pygame
from psychopy import core
# prefs.hardware['audioLib'] = ['pygame']
import time
import os
import numpy as np
import wave
import random
from datetime import date
import pandas as pd
from os.path import join
#from helper_functions import *
# import helper_functions as hf

# In case parallel port needed :
# from psychopy import parallel
# import serial
# from psychopy import visual, core, event, gui, data, sound, parallel, prefs

# GET TIME ELAPSED TASK


from pygame import mixer
pygame.init()


start_time = time.time()



# ----------------- PLAYLISTS for LOCAL/GLOBAL sounds ------------------ #
# ==> Playlists
stim_path = '/Volumes/GoogleDrive/Mon Drive/MVA/S2/Modeling_for_Neuro/Project/phenomenal_quest/stims'
lg_path = os.path.join(os.getcwd(),os.path.join(stim_path,'LG_sounds'))

lg1_playlist = [
    'XXXXY1.wav', 'XXXXY1.wav', 'XXXXY1.wav','XXXXY1.wav', #LS GS
    'XXXXY1.wav', 'XXXXY1.wav', 'XXXXY1.wav','XXXXX1.wav', #LD GD
    'XXXXY1.wav', 'XXXXY1.wav', 'XXXXY1.wav','XXXXY1.wav', #LS GS
    'XXXXY1.wav', 'XXXXY1.wav', 'XXXXY1.wav','XXXXY1.wav', #LS GS
    'XXXXY1.wav', 'XXXXY1.wav', 'XXXXY1.wav','XXXXY1.wav', #LS GS
    'XXXXY1.wav', 'XXXXY1.wav', 'XXXXY1.wav','XXXXY1.wav', #LS GS
    'XXXXY1.wav', 'XXXXY1.wav', 'XXXXY1.wav','XXXXX1.wav', #LD GD
    'XXXXY1.wav', 'XXXXY1.wav', 'XXXXY1.wav','XXXXY1.wav' 
]

lg2_playlist = [
    'XXXXX1.wav', 'XXXXX1.wav', 'XXXXX1.wav','XXXXX1.wav', #LS GS
    'XXXXX1.wav', 'XXXXX1.wav', 'XXXXX1.wav','XXXXY1.wav', #LD GD
    'XXXXX1.wav', 'XXXXX1.wav', 'XXXXX1.wav','XXXXX1.wav', #LS GS
    'XXXXX1.wav', 'XXXXX1.wav', 'XXXXX1.wav','XXXXX1.wav', #LS GS
    'XXXXX1.wav', 'XXXXX1.wav', 'XXXXX1.wav','XXXXX1.wav', #LS GS
    'XXXXX1.wav', 'XXXXX1.wav', 'XXXXX1.wav','XXXXX1.wav', #LS GS
    'XXXXX1.wav', 'XXXXX1.wav', 'XXXXX1.wav','XXXXY1.wav', #LD GD
    'XXXXX1.wav', 'XXXXX1.wav', 'XXXXX1.wav','XXXXX1.wav'
]



# ----------------- PRESETS ------------------ #


# ---> Path for verbal instructions
project_path = os.getcwd()
stims_path = os.path.join(project_path, 'stims')  # contains 'LG_sounds' path


def get_instr_duration(instr_type):
    instr_dic = dict()
    instr_path = os.path.join(stims_path, instr_type + '_sounds')
    instr = os.listdir(instr_path)
    # iterate through instructions
    for i in range(len(instr)):
        stim = os.path.join(instr_path, instr[i])
        with contextlib.closing(wave.open(stim, 'r')) as f:
            frames = f.getnframes()
            rate = f.getframerate()
            instr_duration = frames / float(rate)
            instr_dic.update({instr[i]: instr_duration})
    return instr_dic


def get_playlist(instr_type):
    instr_path = os.path.join(stims_path, instr_type + '_sounds')
    instr = [instruction for instruction in os.listdir(instr_path) if instruction.endswith('.wav')]
    playlist = list()
    for s in range(len(instr)):
        ipath = os.path.join(instr_path, instr[s])
        playlist.append(ipath)
    return playlist


def read_playlist(instr_type, ptime):
    pygame.mixer.pre_init(44100, -16, 2, 32)  # setup mixer to avoid sound lag
    pygame.init()
    #playlist = get_playlist(instr_type)

       # ==> retrieve/reconstruct sounds according to the instr type
    if instr_type == 'LG1':
        playlist = lg1_playlist
    elif instr_type == 'LG2':
        playlist = lg2_playlist

    for i in range(len(playlist)):
        print(playlist[i])
        pygame.mixer.music.load(playlist[i])
        pygame.mixer.music.play()
        while pygame.mixer.music.get_busy():
            pass
        time.sleep(ptime)


def read_conditions(instr_type, timings, sounds, oddballs, n_beeps, n_runs, silence_duration, ptime, start_from_run=0):
    pygame.mixer.pre_init(44100, -16, 2, 32)  # setup mixer to avoid sound lag
    pygame.init()
    pygame.display.set_caption("%s condition " % instr_type)
    screen = pygame.display.set_mode([1536, 864])  # set up drawing window (leave empty for fullscreen mode)
    #bg = pygame.image.load("icm_background.jpeg")  # icm background

    if instr_type != 'oddball':
        n_beeps = 1

    r = 0  # only one run
    sounds_run = np.array(sounds[r])
    playing = start_with_countdown(screen)
    last_time = time.time()
    b = 0

    while b <= n_beeps and playing:
        # while b <= n_beeps :
        screen.blit(bg, (0, 0))
        pygame.display.flip()

        if (time.time() - last_time) > (silence_duration / 1000 - 0.005):  # play sound
            if b < n_beeps:
                if instr_type == 'oddball':
                    pygame.mixer.Sound(sounds_run[b, :].astype('int16')).play()
                else:
                    playlist = get_playlist(instr_type)
                    print(playlist)
                    read_playlist(instr_type, ptime)

                timings[r, b, 0] = pygame.time.get_ticks() / 1000  # record when sound plays

            last_time = time.time()
            b += 1

        for e in pygame.event.get():
            if e.type == pygame.QUIT or (e.type == pygame.KEYDOWN and e.key == pygame.K_ESCAPE):
                print("🛑 Script interrupted by user")
                playing = False
            if e.type == pygame.KEYDOWN and e.key == pygame.K_SPACE and b > 0:  # record when user reacts
                timings[r, b - 1, 1] = pygame.time.get_ticks() / 1000 - timings[r, b - 1, 0]

    # if not playing or end_with_cooldown(screen, r == n_runs-1):
    # break    #wait until next run can start, or press ENTER to skip

    pygame.quit()  # exit application


### HELPER FUNCTIONS 2

In [3]:


"""
n_beeps=30 —Total number of auditory stimuli, per run
n_runs=10 —Total number of runs, per experimental session
beep_duration=50 —Sound duration, in milliseconds
silence_duration=1000 —Interval between sounds, in milliseconds
standard_freq=400 —Main frequency of normal sounds, in Hz
oddball_freq=1000 —Main frequency of odd sounds, in Hz
ratio_oddball=0.2 —Proportion of oddball/standard sounds
"""
n_beeps = 30              #per run
n_runs = 10               #per whole experimental session
beep_duration = 60        #in ms
silence_duration = 1000   #between beeps, in ms
standard_freq = 100       #normal sound frequency, in Hz
oddball_freq = 200       #odd sound frequency, in Hz
ratio_oddball = 0.2       #proportion of oddball/standard sounds

#Generate standard & oddball sounds according to parameters specified, and randomly assign them to each run:
def generate_sounds(n_beeps, n_runs, beep_duration, freq, ratio_oddball, p_id):
    dt = 1/44100
    beep_duration /= 1000
    t_simple = np.arange(dt, beep_duration, dt)
    fade = 0.01
    amplification_f = 16383
    ramp = np.linspace(0, 1, int(fade/dt))
    pad_l = 300
    sound_len = int(beep_duration/dt + pad_l - 1)
    
    s = np.zeros((len(freq), sound_len))
    for sind in range(s.shape[0]):
        sound_temp = np.sin(2*np.pi * freq[sind] * t_simple) * amplification_f
        sound_temp = np.multiply(sound_temp, np.concatenate([ramp, np.ones(len(t_simple) - 2*len(ramp)), ramp[::-1]]))
        s[sind] = np.concatenate([sound_temp, np.zeros(pad_l)])
    
    sounds = np.zeros((n_runs, n_beeps, sound_len))
    oddballs = np.zeros((n_runs, int(n_beeps * ratio_oddball)))
    try: random.seed(int(p_id))  #for reproducibility, set seed to Participant ID
    except: pass

    for r in range(n_runs):
        sounds_prep = np.zeros((n_beeps, sound_len))
        
        iind = np.sort(np.array(random.sample(range(0, n_beeps), int(n_beeps * ratio_oddball))))  #randomize oddball positions
        while ratio_oddball < 0.45 and (any(np.diff(iind) <= 1) or iind[0] < 2):               #ensure no consecutive oddballs
            iind = np.sort(np.array(random.sample(range(0, n_beeps), int(n_beeps * ratio_oddball))))
        
        sounds_prep[::] = s[0]
        sounds_prep[np.array(iind)] = s[1]  #replace the oddball sounds
        sounds[r] = sounds_prep
        
        oddballs[r][::] = iind              #keep track of oddball indexes
    
    oddballs = np.sum(np.eye(n_beeps)[oddballs.astype(int)], 1)   #one-hot representation, useful for performance assessment

    return [sounds, oddballs]
    

#Helper function to wait for user input & add a little "3, 2, 1..." animation:
def start_with_countdown(screen):
    font = pygame.font.SysFont('Consolas', 100)    
    text = 'Press SPACE to start...'.center(25)
    waiting = True
    
    while waiting:
        for e in pygame.event.get():
            if e.type == pygame.KEYDOWN and e.key == pygame.K_SPACE: 
                waiting = False
            if e.type == pygame.QUIT or (e.type == pygame.KEYDOWN and e.key == pygame.K_ESCAPE):
                print("🛑 Script interrupted by user")
                return False
        screen.fill((0, 0, 0))
        screen.blit(font.render(text, True, (80, 80, 80)), (90, 380))
        pygame.display.flip()
    
    pygame.time.set_timer(pygame.USEREVENT, 1000)
    font = pygame.font.SysFont('Consolas', 300)    
    counter, text = 3, '3'.center(1)
    counting = True
    
    while counting:
        for e in pygame.event.get():
            if e.type == pygame.USEREVENT: 
                counter -= 1
                if counter > 0:
                    text = str(counter).center(1)
                else:
                    counting = False
            if e.type == pygame.QUIT or (e.type == pygame.KEYDOWN and e.key == pygame.K_ESCAPE):
                print("🛑 Script interrupted by user")
                return False
        screen.fill((0, 0, 0))
        screen.blit(font.render(text, True, (255, 255, 255)), (680, 300))
        pygame.display.flip()
    
    return True
    

#Helper function to wait at the end of each run, until the next one can start:
def end_with_cooldown(screen, is_last_run = False, cooldown = 5):
    screen.fill((0, 0, 0))
    if is_last_run:
        font = pygame.font.SysFont('Consolas', 80)
        screen.blit(font.render('Thanks for participating :)'.center(30), True, (200, 200, 200)), (100, 380))
    else:
        font = pygame.font.SysFont('Consolas', 100)
        screen.blit(font.render('Nice job!'.center(10), True, (200, 200, 200)), (550, 380))
    pygame.display.flip()

    start_time = pygame.time.get_ticks()
    end_time = pygame.time.get_ticks() + cooldown*1000

    while start_time < end_time:
        start_time = pygame.time.get_ticks()                
        for e in pygame.event.get():
            if e.type == pygame.KEYDOWN and e.key == pygame.K_RETURN:  #cooldown can be skiped by pressing ENTER
                start_time = end_time
            if e.type == pygame.QUIT or (e.type == pygame.KEYDOWN and e.key == pygame.K_ESCAPE):
                print("🛑 Script interrupted by user")
                return True
    return False
 
 
#Main application loop that will instruct the participant to conduct the oddball experiment, and output the results as a pandas dataframe:
def run_session(timings, sounds, oddballs, n_beeps, n_runs, silence_duration, start_from_run = 0):
    pygame.mixer.pre_init(44100, -16, 2, 32)   #setup mixer to avoid sound lag
    pygame.init()
    pygame.display.set_caption("Oddball Session 🕹️")
    screen = pygame.display.set_mode([1536, 864])  #set up drawing window (leave empty for fullscreen mode)
    #bg = pygame.image.load("icm_background.jpeg")       #icm background

    for r in range(start_from_run, n_runs):
        print("Starting run #{}! 🏁      Time: {}".format(r, time.strftime("%H:%M:%S")))
        sounds_run = np.array(sounds[r])

        playing = start_with_countdown(screen)

        last_time = time.time()
        b = 0

        while b <= n_beeps and playing:
            screen.blit(bg, (0,0))
            pygame.display.flip()
            
            if (time.time() - last_time) > (silence_duration/1000 - 0.005):  #play sound
                if b < n_beeps:
                    pygame.mixer.Sound(sounds_run[b, :].astype('int16')).play()
                    timings[r, b, 0] = pygame.time.get_ticks()/1000               #record when sound plays
                    
                last_time = time.time()
                b += 1
            
            for e in pygame.event.get():
                if e.type == pygame.QUIT or (e.type == pygame.KEYDOWN and e.key == pygame.K_ESCAPE):
                    print("🛑 Script interrupted by user")
                    playing = False
                if e.type == pygame.KEYDOWN and e.key == pygame.K_SPACE and b > 0:   #record when user reacts
                    timings[r, b-1, 1] = pygame.time.get_ticks()/1000 - timings[r, b-1, 0]
        
        if not playing or end_with_cooldown(screen, r == n_runs-1): break    #wait until next run can start, or press ENTER to skip

    pygame.quit()  #exit application
    
    results = pd.DataFrame("", index = ["Run " + str(i) for i in range(n_runs)], columns = range(n_beeps))  #create spreadsheet with readable results
    for i in range(start_from_run, r+1):
        for j in range(n_beeps):
            if oddballs[i, j] == 0:
                if timings[i, j, 1] == 0:
                    results.at["Run " + str(i), j] = "-"
                else:
                    results.at["Run " + str(i), j] = "Wrong keypress"
            else:
                if timings[i, j, 1] == 0:
                    results.at["Run " + str(i), j] = "Missed oddball"
                else:
                    results.at["Run " + str(i), j] = str(np.round(timings[i, j, 1], 3)) + " seconds"
    
    return results


# Main application loop that will instruct the participant to conduct the oddball experiment, and output the results as a pandas dataframe:
def run_condition(instr_type_list, pduration):
    pygame.mixer.pre_init(44100, -16, 2, 32)  # setup mixer to avoid sound lag
    pygame.init()
    pygame.display.set_caption("Post-CMD questionnaire")
    screen = pygame.display.set_mode([1536, 864])  # set up drawing window (leave empty for fullscreen mode)
    bg = pygame.image.load("icm_background.jpeg")  # icm background

    for c in range(instr_type_list):
        condition = instr_type_list[c]
        print("Starting %s condition"%condition)

        if condition == 'oddball':
            sounds_run = np.array(sounds[0])

        playing = start_with_countdown(screen)

        last_time = time.time()
        b = 0

        while b <= n_beeps and playing:
            screen.blit(bg, (0, 0))
            pygame.display.flip()

            if (time.time() - last_time) > (silence_duration / 1000 - 0.005):  # play sound
                if b < n_beeps:
                    pygame.mixer.Sound(sounds_run[b, :].astype('int16')).play()
                    timings[r, b, 0] = pygame.time.get_ticks() / 1000  # record when sound plays

                last_time = time.time()
                b += 1

            for e in pygame.event.get():
                if e.type == pygame.QUIT or (e.type == pygame.KEYDOWN and e.key == pygame.K_ESCAPE):
                    print("🛑 Script interrupted by user")
                    playing = False
                if e.type == pygame.KEYDOWN and e.key == pygame.K_SPACE and b > 0:  # record when user reacts
                    timings[r, b - 1, 1] = pygame.time.get_ticks() / 1000 - timings[r, b - 1, 0]

        if not playing or end_with_cooldown(screen,
                                            r == n_runs - 1): break  # wait until next run can start, or press ENTER to skip

    pygame.quit()  # exit application

    results = pd.DataFrame("", index=["Run " + str(i) for i in range(n_runs)],
                           columns=range(n_beeps))  # create spreadsheet with readable results
    for i in range(0, r + 1):
        for j in range(n_beeps):
            if oddballs[i, j] == 0:
                if timings[i, j, 1] == 0:
                    results.at["Run " + str(i), j] = "-"
                else:
                    results.at["Run " + str(i), j] = "Wrong keypress"
            else:
                if timings[i, j, 1] == 0:
                    results.at["Run " + str(i), j] = "Missed oddball"
                else:
                    results.at["Run " + str(i), j] = str(np.round(timings[i, j, 1], 3)) + " seconds"

    return results



In [4]:
# ==> Sound parameters
ptime = 5 # pause time (in sec) between instructions (for non 'oddball' conditions)
n_runs = 1               #per whole experimental session : if 1, len(sounds) = 1 
beep_duration = 90        #in ms
silence_duration = 800   #SOA, in ms
standard_freq = 500       #standard sound frequency, in Hz
oddball_freq = 1000       #deviant sound frequency, in Hz
ratio_oddball = 0.2       #proportion of oddball/standard sounds

n_beeps = 20 #for oddball

var_dict = {}
p_id = 'MJG'

timings = np.zeros((n_runs, n_beeps, 2))  #initialize array to save times when sound played and when participant reacted


# ---> Path for verbal instructions
project_path = os.getcwd()
stims_path = os.path.join(project_path, 'stims')  # contains 'LG_sounds' & 'MCP_sounds' paths



# ----------------- PLAYLISTS ------------------ #
# ==> Playlists
#get_playlist(instr_type)



LG1_playlist = [
    '/Users/romybeaute/Desktop/CMDquest/stims/LG_sounds/XXXXX1.wav',
    '/Users/romybeaute/Desktop/CMDquest/stims/LG_sounds/XXXXX1.wav',
    '/Users/romybeaute/Desktop/CMDquest/stims/LG_sounds/XXXXX1.wav',
    '/Users/romybeaute/Desktop/CMDquest/stims/LG_sounds/XXXXY1.wav', #GD
    '/Users/romybeaute/Desktop/CMDquest/stims/LG_sounds/XXXXX1.wav',
    '/Users/romybeaute/Desktop/CMDquest/stims/LG_sounds/XXXXY1.wav', #GD
    '/Users/romybeaute/Desktop/CMDquest/stims/LG_sounds/XXXXX1.wav',
    '/Users/romybeaute/Desktop/CMDquest/stims/LG_sounds/XXXXX1.wav',
    '/Users/romybeaute/Desktop/CMDquest/stims/LG_sounds/XXXXY1.wav'
]


LG2_playlist = [
    '/Users/romybeaute/Desktop/CMDquest/stims/LG_sounds/XXXXY2.wav',
    '/Users/romybeaute/Desktop/CMDquest/stims/LG_sounds/XXXXY2.wav',
    '/Users/romybeaute/Desktop/CMDquest/stims/LG_sounds/XXXXY2.wav',
    '/Users/romybeaute/Desktop/CMDquest/stims/LG_sounds/XXXXX2.wav', #GD
    '/Users/romybeaute/Desktop/CMDquest/stims/LG_sounds/XXXXY2.wav',
    '/Users/romybeaute/Desktop/CMDquest/stims/LG_sounds/XXXXY2.wav',
    '/Users/romybeaute/Desktop/CMDquest/stims/LG_sounds/XXXXX2.wav',#GD
    '/Users/romybeaute/Desktop/CMDquest/stims/LG_sounds/XXXXY2.wav',
    '/Users/romybeaute/Desktop/CMDquest/stims/LG_sounds/XXXXX2.wav' #GD
]



# ==> Def read_playlist
def read_playlist(playlist, ptime):
    pygame.mixer.pre_init(44100, -16, 2, 32)  # setup mixer to avoid sound lag
    pygame.init()

    for i in range(len(playlist)):
        print(playlist[i])
        pygame.mixer.music.load(playlist[i])
        pygame.mixer.music.play()
        while pygame.mixer.music.get_busy():
            pass
        time.sleep(ptime)
        
# ==> Def read_playlist    
def read_oddball(timings, sounds, oddballs, n_beeps, n_runs, silence_duration, ptime):
    pygame.mixer.pre_init(44100, -16, 2, 32)  # setup mixer to avoid sound lag
    pygame.init()
    r = 0
    sounds_run = np.array(sounds[r])
    last_time = time.time()
    b = 0

    while b <= n_beeps:

        if (time.time() - last_time) > (silence_duration / 1000 - 0.005):  # play sound
            if b < n_beeps:
                pygame.mixer.Sound(sounds_run[b, :].astype('int16')).play()

            last_time = time.time()
            b += 1

    pygame.quit()  # exit application


In [5]:


instr_type = input('Enter instruction type (LG1, LG2, oddball, oddball2 or all): ')

if instr_type == "oddball":
    # ==> generate sounds for oddball
    standard_freq, oddball_freq = 500, 1000
    sounds, oddballs = generate_sounds(n_beeps, n_runs, beep_duration, [standard_freq, oddball_freq], ratio_oddball, p_id)
    read_oddball(timings, sounds, oddballs, n_beeps, n_runs, silence_duration, ptime)
    
elif instr_type == "oddball2":
    # ==> generate sounds for oddball2
    standard_freq, oddball_freq = 400, 150
    beep_duration = 200
    sounds, oddballs = generate_sounds(n_beeps, n_runs, beep_duration, [standard_freq, oddball_freq], ratio_oddball, p_id)
    read_oddball(timings, sounds, oddballs, n_beeps, n_runs, silence_duration, ptime)

else:   
    if instr_type == 'LG1': #LGY2
        playlist = LG1_playlist
        ptime = 0.8
    elif instr_type == 'LG2': #LGX1
        playlist = LG2_playlist
        standard_freq = 250       #standard sound frequency, in Hz
        oddball_freq = 700
        ptime = 0.8
    
    print(playlist)
    read_playlist(playlist, ptime)

Enter instruction type (LG1, LG2, oddball, oddball2 or all): oddball


error: ignored

In [ ]:
# ==> If want to store variables for record of the parameters

variables = ["p_id", "n_runs", "ptime", "n_runs", "beep_duration", "silence_duration", "standard_freq", "ratio_oddball", "n_beeps", "instr_type"]
for var in variables:
    var_dict[var] = eval(var)
file_name = "variables_{}.txt".format(p_id)
with open(file_name, 'w') as file:
    file.write(str(var_dict))
file.close()